In [1]:
GPU_device_id = str(6)
model_id_save_as = 'learningcurve-caednn-full-final'
architecture_id = '../hyperparameter_search/hyperparameter-search-results/CNN-kfoldsfull-final-2-reluupdate_28'
model_class_id = 'CNN1D'
testing_dataset_id = '../../source-interdiction/dataset_generation/validation_dataset_full_200keV_log10time_100.npy'
training_dataset_id = '../../source-interdiction/dataset_generation/training_dataset_full_200keV_log10time_10000.npy'
difficulty_setting = 'full'

train_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 20000,]
earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice
from tensorflow.keras.models import load_model

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)

#### Import model, training function 

In [3]:
from annsa.model_classes import build_cnn_model, compile_model, f1
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [4]:
training_dataset = np.load(training_dataset_id)
training_spectra, training_keys = dataset_to_spectrakeys(training_dataset)

## Load testing dataset

In [5]:
testing_dataset = np.load(testing_dataset_id)
testing_spectra, testing_keys = dataset_to_spectrakeys(testing_dataset)

## Load Model

In [6]:
model_features = load_features(architecture_id)
model_features.output_function = tf.nn.softmax

In [8]:
model_features.cnn_kernels = model_features.cnn_kernel
model_features.pool_sizes = model_features.pool_size
model_features.loss = tf.keras.losses.categorical_crossentropy
model_features.optimizer = tf.keras.optimizers.Adam
model_features.metrics = [f1]
model_features.dropout_rate = model_features.dropout_probability
model_features.output_function = tf.nn.softmax
model_features.input_dim = 1024

In [9]:
cae_model = load_model('./final-models-keras/caepretrain-full-final.hdf5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


## Train network

# Scale input data

In [10]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)
testing_spectra_scaled = model_features.scaler.transform(testing_spectra)

In [11]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1',
    patience=earlystop_patience,
    mode='max',
    min_delta=0.01,
    restore_best_weights=True)

In [12]:
mlb=LabelBinarizer()

for train_size in train_sizes:
    print('\n\nRunning through training size '+str(train_size))
    k_folds_errors = []

    sss = StratifiedShuffleSplit(n_splits=5, train_size=train_size)
    k = 0
    for train_index, _ in sss.split(training_spectra, training_keys):
        print('Running through fold '+str(k))
        training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
        testing_keys_binarized = mlb.transform(testing_keys)
        
        model = compile_model(
            build_cnn_model,
            model_features)
        model_weights = model.get_weights()
        model_weights_updated = model_weights[:]
        model_weights_updated[0:6] = cae_model.get_weights()[0:6]
        model.set_weights(model_weights_updated)

        csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.log')

        output = model.fit(
            x=training_spectra_scaled[train_index],
            y=training_keys_binarized[train_index],
            epochs=num_epochs,
            verbose=1,
            validation_data=(testing_spectra_scaled,
                             testing_keys_binarized),
            shuffle=True,
            callbacks=[earlystop_callback, csv_logger],
        )
        model.save('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.hdf5')
        k += 1




Running through training size 50
Running through fold 0


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 50 samples, validate on 3000 samples
Epoch 1/2000
50/50 [==============================] - 2s 40ms/sample - loss: 3.8979 - f1: 0.0000e+00 - val_loss: 3.8794 - val_f1: 0.0000e+00
Epoch 2/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8739 - f1: 0.0000e+00 - val_loss: 3.8699 - val_f1: 0.0000e+00
Epoch 3/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8504 - f1: 0.0000e+00 - val_loss: 3.8600 - val_f1: 0.0000e+00
Epoch 4/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8203 - f1: 0.0000e+00 - val_loss: 3.8510 - val_f1: 0.0000e+00
Epoch 5/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8069 - f1: 0.0000e+00 - val_loss: 3.8419 - val_f1: 0.0000e+00
Epoch 6/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.7829 - f1: 0.0000e+00 - val_loss: 3.8331 - val_f1: 0.0000e+0

50/50 [==============================] - 0s 6ms/sample - loss: 3.7198 - f1: 0.0000e+00 - val_loss: 3.8229 - val_f1: 0.0000e+00
Epoch 11/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.7163 - f1: 0.0000e+00 - val_loss: 3.8163 - val_f1: 0.0000e+00


Running through training size 100
Running through fold 0
Train on 100 samples, validate on 3000 samples
Epoch 1/2000
100/100 [==============================] - 1s 12ms/sample - loss: 3.9086 - f1: 0.0000e+00 - val_loss: 3.8845 - val_f1: 0.0000e+00
Epoch 2/2000
100/100 [==============================] - 0s 3ms/sample - loss: 3.8783 - f1: 0.0000e+00 - val_loss: 3.8698 - val_f1: 0.0000e+00
Epoch 3/2000
100/100 [==============================] - 0s 3ms/sample - loss: 3.8592 - f1: 0.0000e+00 - val_loss: 3.8540 - val_f1: 0.0000e+00
Epoch 4/2000
100/100 [==============================] - 0s 3ms/sample - loss: 3.8361 - f1: 0.0000e+00 - val_loss: 3.8408 - val_f1: 0.0000e+00
Epoch 5/2000
100/100 [==============================] - 0

Epoch 10/2000
100/100 [==============================] - 0s 4ms/sample - loss: 3.7097 - f1: 0.0000e+00 - val_loss: 3.7576 - val_f1: 0.0000e+00
Epoch 11/2000
100/100 [==============================] - 0s 4ms/sample - loss: 3.6906 - f1: 0.0000e+00 - val_loss: 3.7377 - val_f1: 0.0000e+00


Running through training size 500
Running through fold 0
Train on 500 samples, validate on 3000 samples
Epoch 1/2000
500/500 [==============================] - 2s 3ms/sample - loss: 3.8692 - f1: 0.0000e+00 - val_loss: 3.8208 - val_f1: 0.0000e+00
Epoch 2/2000
500/500 [==============================] - 0s 934us/sample - loss: 3.7856 - f1: 0.0000e+00 - val_loss: 3.7364 - val_f1: 0.0000e+00
Epoch 3/2000
500/500 [==============================] - 0s 907us/sample - loss: 3.7013 - f1: 0.0000e+00 - val_loss: 3.6378 - val_f1: 0.0000e+00
Epoch 4/2000
500/500 [==============================] - 0s 878us/sample - loss: 3.5815 - f1: 0.0000e+00 - val_loss: 3.5163 - val_f1: 0.0000e+00
Epoch 5/2000
500/500 [============

...

15000/15000 [==============================] - 4s 283us/sample - loss: 1.1008 - f1: 0.6952 - val_loss: 1.0242 - val_f1: 0.7330
Epoch 5/2000
15000/15000 [==============================] - 4s 283us/sample - loss: 0.9600 - f1: 0.7569 - val_loss: 0.8977 - val_f1: 0.7829
Epoch 6/2000
15000/15000 [==============================] - 4s 283us/sample - loss: 0.8591 - f1: 0.7966 - val_loss: 0.8248 - val_f1: 0.8071
Epoch 7/2000
15000/15000 [==============================] - 4s 283us/sample - loss: 0.7796 - f1: 0.8249 - val_loss: 0.7595 - val_f1: 0.8267
Epoch 8/2000
15000/15000 [==============================] - 4s 283us/sample - loss: 0.7199 - f1: 0.8429 - val_loss: 0.6963 - val_f1: 0.8464
Epoch 9/2000
15000/15000 [==============================] - 4s 283us/sample - loss: 0.6694 - f1: 0.8599 - val_loss: 0.6596 - val_f1: 0.8607
Epoch 10/2000
15000/15000 [==============================] - 4s 283us/sample - loss: 0.6262 - f1: 0.8741 - val_loss: 0.6278 - val_f1: 0.8646
Epoch 11/2000
15000/15000 [=====

Epoch 19/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.4293 - f1: 0.9239 - val_loss: 0.4510 - val_f1: 0.9174
Epoch 20/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.4175 - f1: 0.9269 - val_loss: 0.4455 - val_f1: 0.9127
Epoch 21/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.4053 - f1: 0.9296 - val_loss: 0.4317 - val_f1: 0.9168
Epoch 22/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.3908 - f1: 0.9330 - val_loss: 0.4199 - val_f1: 0.9176
Epoch 23/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.3794 - f1: 0.9350 - val_loss: 0.4131 - val_f1: 0.9239
Epoch 24/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.3714 - f1: 0.9376 - val_loss: 0.4172 - val_f1: 0.9209
Epoch 25/2000
15000/15000 [==============================] - 4s 284us/sample - loss: 0.3628 - f1: 0.9401 - val_loss: 0.4137 - val_f1: 0.9228
Epoch 26/2000

20000/20000 [==============================] - 6s 279us/sample - loss: 0.2503 - f1: 0.9652 - val_loss: 0.2995 - val_f1: 0.9461
Epoch 35/2000
20000/20000 [==============================] - 6s 279us/sample - loss: 0.2450 - f1: 0.9667 - val_loss: 0.2973 - val_f1: 0.9522
Running through fold 1
Train on 20000 samples, validate on 3000 samples
Epoch 1/2000
20000/20000 [==============================] - 8s 419us/sample - loss: 2.5667 - f1: 0.1197 - val_loss: 1.7918 - val_f1: 0.2939
Epoch 2/2000
20000/20000 [==============================] - 6s 279us/sample - loss: 1.5176 - f1: 0.4576 - val_loss: 1.2889 - val_f1: 0.6077
Epoch 3/2000
20000/20000 [==============================] - 6s 281us/sample - loss: 1.1569 - f1: 0.6608 - val_loss: 1.0226 - val_f1: 0.7410
Epoch 4/2000
20000/20000 [==============================] - 6s 280us/sample - loss: 0.9543 - f1: 0.7632 - val_loss: 0.8851 - val_f1: 0.7804
Epoch 5/2000
20000/20000 [==============================] - 6s 279us/sample - loss: 0.8311 - f1: 0.8

20000/20000 [==============================] - 6s 280us/sample - loss: 0.3747 - f1: 0.9350 - val_loss: 0.4013 - val_f1: 0.9268
Epoch 19/2000
20000/20000 [==============================] - 6s 281us/sample - loss: 0.3659 - f1: 0.9357 - val_loss: 0.3957 - val_f1: 0.9253
Epoch 20/2000
20000/20000 [==============================] - 6s 281us/sample - loss: 0.3557 - f1: 0.9377 - val_loss: 0.3718 - val_f1: 0.9361
Epoch 21/2000
20000/20000 [==============================] - 6s 281us/sample - loss: 0.3431 - f1: 0.9415 - val_loss: 0.3829 - val_f1: 0.9305
Epoch 22/2000
20000/20000 [==============================] - 6s 281us/sample - loss: 0.3325 - f1: 0.9431 - val_loss: 0.3652 - val_f1: 0.9293
Epoch 23/2000
20000/20000 [==============================] - 6s 281us/sample - loss: 0.3247 - f1: 0.9447 - val_loss: 0.3586 - val_f1: 0.9319
Epoch 24/2000
20000/20000 [==============================] - 6s 281us/sample - loss: 0.3151 - f1: 0.9464 - val_loss: 0.3538 - val_f1: 0.9359
Epoch 25/2000
20000/20000 [

20000/20000 [==============================] - 6s 285us/sample - loss: 0.4679 - f1: 0.9152 - val_loss: 0.4693 - val_f1: 0.9180
Epoch 14/2000
20000/20000 [==============================] - 6s 286us/sample - loss: 0.4438 - f1: 0.9209 - val_loss: 0.4462 - val_f1: 0.9176
Epoch 15/2000
20000/20000 [==============================] - 6s 286us/sample - loss: 0.4232 - f1: 0.9258 - val_loss: 0.4305 - val_f1: 0.9266
Epoch 16/2000
20000/20000 [==============================] - 6s 286us/sample - loss: 0.4062 - f1: 0.9308 - val_loss: 0.4223 - val_f1: 0.9241
Epoch 17/2000
20000/20000 [==============================] - 6s 286us/sample - loss: 0.3894 - f1: 0.9333 - val_loss: 0.4037 - val_f1: 0.9302
Epoch 18/2000
20000/20000 [==============================] - 6s 286us/sample - loss: 0.3782 - f1: 0.9341 - val_loss: 0.3946 - val_f1: 0.9305
Epoch 19/2000
20000/20000 [==============================] - 6s 286us/sample - loss: 0.3619 - f1: 0.9396 - val_loss: 0.3802 - val_f1: 0.9324
Epoch 20/2000
20000/20000 [